# A Movie behind a Script


In [1]:
import os
import re
import findspark
import pandas as pd
findspark.init()
from pyspark.sql import *
from pyspark.sql.functions import unix_timestamp, udf, to_date
from pyspark.sql.types import *
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# import gzip
import urllib.request

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'UTC')
sc = spark.sparkContext

## Overview of dataset

(Considering only the folder OpenSubtitles2018) The dataset presented consists of 31GB of xml files where we can find the subtitles in different languages of movies, tv shows and trailers. The data is separeted in different folders, first separating the subtitles by language, then by year and finally by identifier. Each Id is supposed to be its IMDb identifier but this needs to be checked. There are some year folders however that are not indicative, we can find folder 0. We can also notice that per film we can have multiple subtitle xml files. The decompressed xml files vary a great deal in size aswell, we can have 9000KB file and 5KB.

Each xml file has a document id and contains the following metadata splitted in 3 different categories: Conversion, Source, and Subtitle.

Coversion contains:
- Number of Sentences
- Number of corrected words
- Number of unknown words
- Number of tokens
- Encoding type

Source contains:
- Genre (Action, drama, horror, etc. Can have multiple)
- Year

Subtitle contains:
- Language
- Date (creation of file or release date of associated video?) -> XML file
- Duration (of video associated)
- Cds (can be 1/5, 2/3)
- Blocks
- Confidence

We can use these metadata to find different statistics that might reveal interesting information.

For the actual subtitles in each xml file we can see that they are stored in sentences, each one having an unique id(integers in increasing order starting at 1). Each sentence has a set timestamp and a set of words. Every timestamp and word has also an id, the timestamp id has two different formats. 

First for the timestamp we have"T#S", "T#E" where # is an increasing integer, "S" indicates start and "E" indicates end. The words inbetween a start and end of timestamp are shown on the screen during the time indicated by the time stamp. **This is a great indicator of fast dialog!** 

For the words the id is simply an increasing number of decimal numbers "X.Y" where X is the string id and Y is the word id within the corresponding string.

In [3]:
baseURL = "https://datasets.imdbws.com/"
ratings_fn = "title.ratings.tsv.gz"
basics_fn = "title.basics.tsv.gz"

In [4]:
df_ratings = pd.read_csv(baseURL + ratings_fn, sep='\t', compression='gzip')
df_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.8,1439
1,tt0000002,6.3,172
2,tt0000003,6.6,1040
3,tt0000004,6.4,102
4,tt0000005,6.2,1735


In [ ]:
df_basics = pd.read_csv(baseURL + basics_fn, sep='\t', compression='gzip')
df_basics.head()

In [ ]:
df_sample_film = sqlContext.read.format('com.databricks.spark.xml').options(rowTag='s') \
                        .load('data_subtitles/2017/331314/6908253.xml/')

In [ ]:
df_sample_film.printSchema()